## sentdex - 딥러닝 Pt.9

<h3> 목표 : 정규화 및 시퀀스 만들기</h3>
<hr>

In [1]:
# 60분을 통해 3분 (미래) 예측

SEQ_LEN = 60
FUTURE_PERIOD_PREDICT = 3
RATIO_TO_PREDICT = "LTC-USD"

def classify(current, future):
    if float(future) > float(current):
        return 1
    else:
        return 0
    
import pandas as pd
import os

main_df = pd.DataFrame()

ratios = ["BTC-USD", "LTC-USD", "ETH-USD", "BCH-USD"]
for ratio in ratios:
    dataset = f"crypto_data/{ratio}.csv"
    
    df = pd.read_csv(dataset,
                     names = ["time", "low", "high", "open", "close", "volume"])
    df.rename(columns = {"close" : f"{ratio}_close", "volume" : f"{ratio}_volume"}, inplace = True)
    
    df.set_index("time", inplace = True)
    df = df[[f"{ratio}_close", f"{ratio}_volume"]]
    
    if len(main_df) == 0:
        main_df = df
    else:
        main_df = main_df.join(df)
        
# "future" 값은 사실 "close"가 3일치 밀린 (shifted) 값이다!
main_df["future"] = main_df[ f"{RATIO_TO_PREDICT}_close" ].shift(-FUTURE_PERIOD_PREDICT)        

# 분류합시다 (ML은 아니지만)
main_df['target'] = list(map(classify, main_df[ f"{RATIO_TO_PREDICT}_close"], main_df["future"]))

print(main_df[ [ f"{RATIO_TO_PREDICT}_close", "future", "target"] ].head(10))

            LTC-USD_close     future  target
time                                        
1528968660      96.580002  96.500000       0
1528968720      96.660004  96.389999       0
1528968780      96.570000  96.519997       0
1528968840      96.500000  96.440002       0
1528968900      96.389999  96.470001       1
1528968960      96.519997  96.400002       0
1528969020      96.440002  96.400002       0
1528969080      96.470001  96.400002       0
1528969140      96.400002  96.400002       0
1528969200      96.400002  96.400002       0


<hr>
... <strong>지금까지는 pt.8에서 했죠</strong>

In [22]:
!pip install sklearn

** **RNN에서 진심 중요한 파트임** **
- **시퀀셜 데이터 구성**

In [2]:
from sklearn import preprocessing
from collections import deque
import random
import numpy as np

def preprocess_df(df):
    
    # "future" 열은 이제 필요 없음
    df = df.drop("future", axis = 1)
    
    # scaling
    for col in df.columns:
        if col != "target":
            
            df[col] = df[col].pct_change()
            df.dropna(inplace = True)
            df[col] = preprocessing.scale(df[col].values)
    
    # 혹시 모르니까..
    df.dropna(inplace = True)
    
    
    sequential_data = []
    prev_mins = deque(maxlen = SEQ_LEN)
    
    for i in df.values:  # df.values: df의 각 행
        prev_mins.append([n for n in i[:-1]])  # why i[:-1]? : target class를 미포함시키기 위해서
        
        if len(prev_mins) == SEQ_LEN:
            sequential_data.append([np.array(prev_mins), i[-1]])  # 지난 60일 ( (i)일 기준 ) + _일의 타겟값(i)
            
    random.shuffle(sequential_data)


**Validation과 아닌 놈의 기준을 두자.**

In [3]:
times = sorted(main_df.index.values)
last_5pct = times[-int(0.05 * len(times))] # threshold
print(last_5pct)

1534922100


**Validation 스플릿! (5%)**

In [4]:
validation_main_df = main_df[ (main_df.index >= last_5pct)] # validation (5%)
main_df = main_df[ (main_df.index < last_5pct)] # the rest

In [5]:
#train_x, train_y = preprocess_df(main_df)
#validation_x, validation_y = preprocess_df(validation_main_df)